In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
from bokeh.io import output_notebook
output_notebook()
from bokeh.layouts import row, column, widgetbox
from bokeh.plotting import figure, output_notebook, output_file, show
from bokeh.models.sources import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.models import OpenURL, TapTool, CustomJS, Slider, Range1d, DatetimeTickFormatter
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from bokeh.charts import Histogram, Bar

Loading BokehJS ...

In [33]:
output_cols = [
    'out_objectness', 
    'out_count', 
    'out_max',
    'out_diversity',
    
    #'objectness',
    
    'inout_noise',
    'in_objectness', 
    'in_count', 
    'in_max',
    'in_diversity'
]
pd.options.display.max_columns = 1000


df = pd.read_csv('hypers.csv', index_col=0)
#jobset88 = without sparsity bug
#jobset89 = with sparsity bug
df = df[df['jobset'] == 'jobset88']
df = df[df['nb_active']==30]
df = df[['id'] + output_cols + ['topology']]
df['value'] = df['out_objectness'] + (10 * df['out_count'] + 10 * df['out_diversity'])
df['link'] = df.apply(lambda mo:'http://134.158.75.73:24000/pattern/out_of_class_figs.*{}.*'.format(mo['id']), axis=1)

In [47]:
df['line_color'] = 'royalblue'
df.loc[df['topology'] == 'diamond', 'line_color'] = 'coral'
df.loc[df['topology'] == 'hourglass', 'line_color'] = 'seagreen'
df.loc[df['topology'] == 'parallel', 'line_color'] = 'goldenrod'
df.loc[df['topology'] == 'trumpet', 'line_color'] = 'palevioletred'
df.loc[df['topology'] == 'eiffel', 'line_color'] = 'royalblue'

In [49]:
x = 'out_count'
y = 'out_diversity'
output_file('models_{}_{}.html'.format(x.replace('out_', ''), y.replace('out_', '') ))

hover = HoverTool(
        tooltips=[
            ("model", "@topology"),
        ]
    )

tools = [hover, 'pan,wheel_zoom,box_zoom,reset,previewsave,tap']
p = figure(
    plot_width=900, plot_height=900, tools=tools, title='',
)

p.circle(
    'out_objectness', 'out_count', size=10, line_color='line_color', fill_color='line_color',
    fill_alpha=0.9, line_width=1, source=ColumnDataSource(df), legend='topology'
)

p.xaxis.axis_label = x
p.yaxis.axis_label = y
p.xaxis.axis_label_text_font_size = '14pt'
p.yaxis.axis_label_text_font_size = '14pt'
p.legend.label_text_font_size = '14pt'
p.title.text_font_size = '16pt'
p.xaxis.major_label_text_font_size = '10pt'

url = "@link"
taptool = p.select(type=TapTool)
taptool.callback = OpenURL(url=url)

show(p)